# bqplot

Welcome to bqplot! This notebook will serve as an introduction to bqplot and the widgets framework of the Jupyter Notebook. 

### So, what is bqplot?

`bqplot` is a Grammar of Graphics based interactive visualization library for the Jupyter notebook.

Uhh, so - what does that mean?

In [3]:
## Let's see!

In [138]:
# Let's begin by importing some libraries we'll need
import numpy as np
import pandas as pd
import os
from __future__ import print_function # So that this notebook becomes both Python 2 and Python 3 compatible

# And creating some random data
size = 100
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * 100.0)

In [27]:
# We'll start with bqplot's matplotlib inspired API

from bqplot import pyplot as plt

It means that any plot you generate is, by default, an interactive web element that you can control using **only** Python.

In [76]:
figure = plt.figure(title='My First Plot')
scatter = plt.scatter(x_data, y_data)
plt.show()

It also means that any aspect of the plot **is an interactive widget**. This allows us to change any aspect of the plot - after it's been drawn.

#### Try it! Run the cell below a few times and watch the plot above change!

In [96]:
scatter.y = np.cumsum(np.random.randn(size)  * 100.0)

Since a bqplot chart is a web element, we should be able to animate it right?

### Let's animate it!

In [92]:
figure.animation_duration = 500

Voila!

In [ ]:
scatter.y = np.cumsum(np.random.randn(size)  * 100.0)

That doesn't apply to just the x and y. To have a look at the attributes, take a look at the documentation!

In [30]:
# Say, the color
scatter.colors = ['Red']

In [34]:
# Or, the marker style
scatter.marker = 'diamond'

It's important to remember that an interactive widget means that the `JavaScript` and the `Python` communicate. So, the plot can be changed through a single line of python code, or a piece of python code can be triggered by a change in the plot. Let's go through a simple example. Say we have a function `foo`:

In [35]:
def foo(change):
    print('This is a trait change. Foo was called by the fact that we moved the Scatter')
    print('In fact, the Scatter plot sent us all the new data: ')
    print('To access the data, try modifying the function and printing the data variable')

We can call `foo` everytime any attribute of our scatter is changed. Say, the `y` values:

In [37]:
# First, we hook up our function `foo` to the colors attribute (or Trait) of the scatter plot
scatter.observe(foo, 'y')

To allow the points in the `Scatter` to be moved interactively, we set the `enable_move` attribute to `True`

In [40]:
scatter.enable_move = True

Go ahead, head over to the chart and move any point in some way. This move (which happens on the `JavaScript` side should trigger our `Python` function `foo`.

## That's great, but what does that have to do with the Grammar of Graphics?

`bqplot` has two different APIs. One is the matplotlib inspired `pyplot` which we used above (you can think of it as similar to `qplot` in `ggplot2`). The other one, the verbose API, is meant to expose every element of a plot individually, so that their attriutes can be controlled in an atomic way. In order to truly use `bqplot` to build complex and feature-rich GUIs, it pays to understand the underlying theory that is used to create a plot.

**A `Scale` is a mapping from (function that converts) data coordinates to figure coordinates.** What this means is that, a `Scale` takes a set of values in any arbitrary unit (say number of people, or $, or litres) and converts it to pixels (or colors for a `ColorScale`).

In [42]:
# First, we import the scales
from bqplot import LinearScale

In [43]:
# Let's create a scale for the x attribute, and a scale for the y attribute
x_sc = LinearScale()
y_sc = LinearScale()

Now, we need to create the actual `Mark` that will visually represent the data. Let's pick a `Scatter` chart to start.

In [44]:
from bqplot import Scatter

scatter_chart = Scatter(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc})

Most of the time, the actual `Figure` co-ordinates don't really mean anything to us. So, what we need is the visual representation of our `Scale`, which is called an `Axis`.

In [46]:
from bqplot import Axis

x_ax = Axis(label='X', scale=x_sc)
y_ax = Axis(label='Y', scale=y_sc, orientation='vertical')

And finally, we put it all together on a canvas, which is called a `Figure`.

In [48]:
from bqplot import Figure

fig = Figure(marks=[scatter_chart], title='A Figure', axes=[x_ax, y_ax])
fig

Now, that the plot has been generated, we can control every single attribute of it. Let's say we wanted to color the chart based on some other data.

In [49]:
# First, we generate some random color data.
color_data = np.random.randint(0, 2, size=100)

Now, we define a ColorScale to map the color_data to actual colors

In [51]:
from bqplot import ColorScale

# The colors trait controls the actual colors we want to map to. It can also take a min, mid, max list of
# colors to be interpolated between for continuous data.
col_sc = ColorScale(colors=['MediumSeaGreen', 'Red'])

In [52]:
scatter_chart.scales = {'x': x_sc, 'y': y_sc, 'color': col_sc}
# We pass the color data to the Scatter Chart through it's color attribute
scatter_chart.color = color_data

The grammar of graphics framework allows us to overlay multiple visualizations on a single `Figure` by having the visualization share the `Scales`. So, for example, if we had a `Bar` chart that we would like to plot alongside the `Scatter` plot, we just pass it the same `Scales`.

In [53]:
from bqplot import Bars

new_size = 50
scale = 100.
x_data_new = np.arange(new_size)
y_data_new = np.cumsum(np.random.randn(new_size)  * scale)

In [54]:
# All we need to do to add a bar chart to the Figure is pass the same scales to the Mark
bar_chart = Bars(x=x_data_new, y=y_data_new, scales={'x': x_sc, 'y': y_sc})

Finally, we add the new `Mark` to the `Figure` to update the plot!

In [55]:
fig.marks = [scatter_chart, bar_chart]

## Overview of bqplot `Marks` 

Checkout the [bqplot examples](www.github.com/bloomberg/bqplot/examples/) for a simple walkthrough of all the basic visualizations that bqplot offers

## Interactions

### Tooltips

Tooltips can be made to display an attribute of the data:

In [139]:
county_data = pd.read_csv(os.path.abspath('data_files/2008-election-results.csv'))
winner = np.array(['McCain'] * county_data.shape[0])
winner[(county_data['Obama'] > county_data['McCain']).values] = 'Obama'

In [140]:
from bqplot import Tooltip, AlbersUSA, OrdinalColorScale, ColorAxis

In [147]:
sc_geo_county = AlbersUSA()
sc_c1_county = OrdinalColorScale(domain=['McCain', 'Obama'], colors=['Red', 'DeepSkyBlue'])
color_data_county = dict(zip(county_data['FIPS'].values.astype(int), list(winner)))

map_styles_county = {'color': color_data_county,
              'scales': {'projection': sc_geo_county, 'color': sc_c1_county}, 'colors': {'default_color': 'Grey'}}

county_fig = plt.figure(title='US Elections 2008 - Example')
county_map = plt.geo(map_data='USCountiesMap', **map_styles_county)

In [148]:
map_tt = Tooltip(fields=['id', 'name', 'color'], labels=['County Code', 'County Name', 'Winner'])

In [149]:
county_map.tooltip = map_tt

In [150]:
county_fig

Alternately, any instance of a widget can be made a `Tooltip`

In [113]:
tt_fig = plt.figure(title='It works!')
tt_line = plt.plot(x_data, y_data)
county_map.tooltip = tt_fig

**Exercise**: Use the `on_hover` function provided by the `Map` class to generate a specific tooltip for each country.

## Selections

In [130]:
fig = plt.figure(title='My First Plot')
scatter_plot = plt.scatter(x_data, y_data, interactions={'click': 'select'},
                           unselected_style={'opacity': 0.4})
plt.show()

The points selected are indexes []
[[33.306341911764704, 850.4543074479469], [36.21810661764705, 855.3423881964055]]
[[33.306341911764704, 698.9238042457295], [53.49457720588235, 855.3423881964055]]
The points selected are indexes [42, 46, 48]
[[33.306341911764704, 669.5953197549776], [57.1828125, 855.3423881964055]]
The points selected are indexes [42, 46, 48, 49]
[[33.306341911764704, 630.4906737673086], [60.676930147058826, 855.3423881964055]]
The points selected are indexes [42, 46, 48, 49, 51]
[[33.306341911764704, 620.7145122703914], [61.06516544117647, 855.3423881964055]]
[[33.306341911764704, 606.0502700250155], [61.45340073529411, 855.3423881964055]]
The points selected are indexes [42, 46, 48, 49, 50, 51, 52, 55, 56, 57]
[[33.306341911764704, 469.184009068174], [64.94751838235294, 855.3423881964055]]
The points selected are indexes [42, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
[[33.306341911764704, 386.08663634437744], [66.50045955882354, 855.3423881964055]]
The points

`selected` is a trait. So we can always observe it.

In [131]:
def observe_selected(change):
    print('The points selected are indexes {}'.format(scatter_plot.selected))
    
scatter_plot.observe(observe_selected, 'selected')

In [132]:
def call_back(name, value):
    print(value)
    
plt.brush_selector(call_back)

## Mixing bqplot with ipywidgets

The entire framework for bqplot is based on `d3js` (on the JavaScript side) and on the [ipywidgets](www.github.com/ipywidgets) framework. This means that integrating it with native Jupyter widgets is a seamless experience. As is integrating it with other widget based libraries, such as:

- ipyleaflet
- ipyvolume
- pythreejs